In [ ]:
#run this first before reading any data otherwise you get an error 
import pandas as pd
import glob
import warnings
from google.colab import drive 

mount_path = '/content/drive'

drive.mount(mount_path, force_remount=True)

Mounted at /content/drive


In [ ]:
def standardize_data(path = '/content/drive/Shareddrives/MATH456/FRED CSVs'):
  files = glob.glob(path + "/*.csv")

  complete_data = []

  #this loop combines all of the data together into a single csv file
  for single_file in files:
      df = pd.DataFrame(pd.read_csv(single_file))

      #retrieves name of the variable we want
      variable = list(df.columns)[1]
      df['DATE'] = pd.to_datetime(df['DATE'])

      #since not all of the data is the same, we had figured that standardizing
      #it to quarterly averages will be the best way to attack this issue
      df['quarter'] = df['DATE'].dt.to_period('Q')

      #this gets the name of the numeric dataset
      df = df[df[variable] != '.']

      #good data practice to convert calculation data into numerical values
      #as these were string data before
      df[variable] = pd.to_numeric(df[variable])
      
      #we only want data from the latest first date in each dataset to 
      #earliest end date in each dataset. will be changed if needed
      df = df.loc[df['DATE'].between('2012-10-31','2022-04-01', inclusive=True)]
      complete_data.append(df.groupby(['quarter'])[variable].mean().round(2))

  complete_set = pd.concat(complete_data, axis=1).fillna(0)
  #wrote to new file outside of where the raw datasets were
  #this is helpful in case we need to make changes to dates needed in future 
  complete_set.to_csv(path.strip('FRED CSVs') + 'Cleaned.csv')
  warnings.filter("ignore")

  return 'Done.'

standardize_data()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
